In [1]:
import numpy as np
import pygame
import sys

# import imageio

from quaternion import Point3D, Rotation, Vecteur3D

from drawing_obj import Point_obj, Groupe_drawable_object, Screen, Vecteur_obj, Droite_obj, Circle_obj, Sphere_obj, Tore_obj

%load_ext autoreload
%autoreload 2

pygame 2.6.1 (SDL 2.28.4, Python 3.12.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
T = Tore_obj( Vecteur3D( Point3D(0,0,3), Point3D(0,0,2) ), 3, 0.75, 20 )


In [4]:
points = [
    Point_obj(x, y, z)
    for x in (1.,3.) for y in (-1.,1.) for z in (-1.,1.)
]

cube_point = points[0]
for p in points[1:]:
    cube_point = cube_point | p

d1 = Droite_obj( Point3D([3],[1],[1]), Point3D([1],[1],[1,]) )
# d2 = Droite_obj( Point3D([3],[-1],[1]), Point3D([1],[-1],[1,]) )

# dd = d1 | d1 | d1

# d = Droite_obj( Point3D([3,3,3,3],[1,-1.,1,-1],[1,1,-1,-1]), Point3D([1,1,1,1],[1,-1,1,-1],[1,1,-1,-1]) )
# d = d | Droite_obj( Point3D([1,3,3,1],[-1,-1,-1,-1],[1,1,-1,-1]), Point3D([1,3,3,1],[1,1,1,1],[1,1,-1,-1]) )


for count, i in enumerate(np.linspace( -40,40, 21 )):
    if count:
        grille = grille | Droite_obj( Point3D(1,i,0.), Point3D(0.,i,0.) )
        grille = grille | Droite_obj( Point3D(i,1.,0.), Point3D(i,0.,0.) )
    else:
        grille = Droite_obj( Point3D(1,i,0.), Point3D(0.,i,0.) )
        grille = grille | Droite_obj( Point3D(i,1.,0.), Point3D(i,0.,0.) )

#grille = Droite_obj( Point3D(1,16,0.), Point3D(0.,16,0.) )
#grille = grille | Droite_obj( Point3D(1,-16,0.), Point3D(0.,-16,0.) )

# grille = [ Droite_obj( Point3D(1,i,0.), Point3D(0.,i,0.) ) for i in np.linspace( -3,3, 7 )]
# grille += [ Droite_obj( Point3D(i,1.,0.), Point3D(i,0.,0.) ) for i in np.linspace( -3,3, 11 )]

# vs  = [ Vecteur3D( Point3D(0.,i,0.), Point3D(1,i,0.) ) for i in np.linspace( -3,3, 7 )]
# vs += [ Vecteur3D( Point3D(i,0.,0.), Point3D(i,1.,0.) ) for i in np.linspace( -3,3, 7 )]

vss = Vecteur_obj( Point3D(0.,0.,0.), Point3D(1,0,0.) )
for i in np.linspace( -3,3, 7 ):
    vss = vss | Vecteur_obj( Point3D(0.,i,0.), Point3D(1,i,0.) )
    vss = vss | Vecteur_obj( Point3D(i,0.,0.), Point3D(i,1.,0.) )

#vss = Vecteur3D( Point3D(0.,0.,0.), Point3D(3,0,0.) )


C = Circle_obj( Vecteur3D(Point3D(1,0,0), Point3D(0,0,0)).to_unitaire, 5, 20  )

sph = Sphere_obj( Point3D(1,1,1), 1., 32 )

T = Tore_obj( Vecteur3D( Point3D(0,0,3), Point3D(0,0,2) ), 3, 0.75, 20 )

grp = Groupe_drawable_object()
grp.add_in( cube_point )
#grp.add_in( d1 )
grp.add_in( grille )
#grp.add_in( vss )
grp.add_in(sph)
grp.add_in(C)

grp.add_in(T)

(48, 3)


In [5]:
SIZE, FPS = 600, 30
S = Screen(2., 110, size_pixel=SIZE, DoV=10.)

pygame.init()
screen = pygame.display.set_mode((SIZE, SIZE))
clock = pygame.time.Clock()

### open a writer 
# writer = imageio.get_writer("recording.mp4", fps=30)

alpha_speed_translation = 0.1
alpha_speed_rot = 3
d_angle = np.arctan( S.demi_screen_size / 1.5 ) / (SIZE/2)  * alpha_speed_rot ### rad/px
rot_y = Rotation(d_angle, Vecteur3D( Point3D(2.,0.,1.), Point3D(2.,0.,0.)) )
rot_z = Rotation(d_angle, Vecteur3D( Point3D(2.,1.,0.), Point3D(2.,0.,0.)) )

rot_y_view = Rotation(d_angle, Vecteur3D( Point3D(0.,0.,1.) ) )
rot_z_view = Rotation(d_angle, Vecteur3D( Point3D(0.,1.,0.) ) )

### 
rot_cercle = Rotation(d_angle, C.vecteur_normal.copy )
C.add_rot( rot_cercle )
### 
rot_sphere = Rotation(d_angle, Vecteur3D( Point3D(9,3,7), sph.centre.copy)  )
sph.add_rot( rot_sphere )
### 
rot_torus = Rotation(d_angle, T.vecteur_normal.copy )
T.add_rot(rot_torus)


# Main loop
running = True
dragging = False
dyr, dzr = 0, 0
while running:
    for e in pygame.event.get():
        if e.type == pygame.QUIT:
            running = False

        ### Translations
        keys = pygame.key.get_pressed()
        if keys[pygame.K_z]:
            grp >>= (Vecteur3D( Point3D(-1.,0.,0.) ) * alpha_speed_translation)
        if keys[pygame.K_s]:
            grp >>= (Vecteur3D( Point3D(1.,0.,0.) )  * alpha_speed_translation)
        if keys[pygame.K_q]:
            grp >>= (Vecteur3D( Point3D(0.,-1.,0.) ) * alpha_speed_translation)
        if keys[pygame.K_d]:
            grp >>= (Vecteur3D( Point3D(0.,1.,0.) )  * alpha_speed_translation)
        if keys[pygame.K_SPACE]:
            grp >>= (Vecteur3D( Point3D(0.,0.,-1.) ) * alpha_speed_translation)
        if keys[pygame.K_LCTRL]:
            grp >>= (Vecteur3D( Point3D(0.,0.,1.) )  * alpha_speed_translation)

        left, middle, right = pygame.mouse.get_pressed()
        ### rotations
        if right and e.type == pygame.MOUSEMOTION:
            dyr, dzr = e.rel
            rot_y_view.angle = d_angle * dyr
            grp *= rot_y_view
            rot_z_view.angle = d_angle * (-dzr)
            grp *= rot_z_view
        elif left and e.type == pygame.MOUSEMOTION:
            dyr, dzr = e.rel
            rot_y.angle = d_angle * dyr
            grp *= rot_y
            rot_z.angle = d_angle * (-dzr)
            grp *= rot_z

    grp.apply_rotation_propre()

            
    screen.fill((20,20,20))

    ### Draw ojects on screen
    for obj in grp.list_of_obj:

        i, j = obj.project_on_screen(S)

        if i.ndim==1:
            for ii, jj in zip(i,j):
                pygame.draw.circle(screen, (250,250,250), (jj, ii), 2.2)

        elif i.ndim==2:
            for ii, jj in zip( i.T, j.T ): 
                pygame.draw.line(screen, (250,250,250), (jj[0],ii[0]), (jj[1],ii[1]), 1)

        else:
            print( f"Drawing of obj {type(obj)} not inplemented" )

    pygame.display.flip()


    # # capture frame
    # frame = pygame.surfarray.array3d(screen)       # shape (W,H,3)
    # frame = np.transpose(frame, (1, 0, 2))         # to (H,W,3)
    # writer.append_data(frame)

    dt = clock.tick(FPS)
    if dt>=40:
        print( f"ping : {dt} ms" )

pygame.quit()
sys.exit()

/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:288: RuntimeWarning: invalid value encountered in sqrt
  tpos = np.sqrt(
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:365: RuntimeWarning: divide by zero encountered in divide
  return (-f_ * ca + f_ * cs - cs * xa_) / (cs * Vx_ + f_ * Vc_)
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:358: RuntimeWarning: invalid value encountered in multiply
  return -f_ * (Vc_ * ts + ca) / (-f_ + Vx_ * ts + xa_)
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:358: RuntimeWarning: invalid value encountered in divide
  return -f_ * (Vc_ * ts + ca) / (-f_ + Vx_ * ts + xa_)
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:372: RuntimeWarning: invalid value encountered in multiply
  x3d1 = Ax + Vx * t1
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:373: RuntimeWarning: invalid value encountered in multiply
  y3d1 = Ay + Vy * t1
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:374: RuntimeWarning: invalid 

SystemExit: 

/home/nicolas-gillet/micromamba/envs/generale/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
